### Importing the relevant libraries and Loading the Data

In [11]:

import numpy as np
from sklearn import preprocessing
import tensorflow as tf
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]


### Balancing the dataset

In [12]:
#shuffle once before balancing
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

### Standardising the data

In [13]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffling the data

In [14]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Splitting the dataset into train, validation, and test

In [15]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1786.0 3579 0.4990220732048058
229.0 447 0.5123042505592841
222.0 448 0.4955357142857143


### Saving the three datasets in *.npz

In [16]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Splitting the Data into Inputs and targets

In [17]:

npz = np.load('Audiobooks_data_train.npz')


train_inputs = npz['inputs'].astype(np.float)

train_targets = npz['targets'].astype(np.int)


npz = np.load('Audiobooks_data_validation.npz')

validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)


npz = np.load('Audiobooks_data_test.npz')

test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [18]:

input_size = 10
output_size = 2

hidden_layer_size = 50
    

model = tf.keras.Sequential([
  
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
  
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])




batch_size = 100


max_epochs = 100


early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)


model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2
          )  

Epoch 1/100
36/36 - 1s - loss: 0.5708 - accuracy: 0.7136 - val_loss: 0.5015 - val_accuracy: 0.7136
Epoch 2/100
36/36 - 0s - loss: 0.4494 - accuracy: 0.7773 - val_loss: 0.4579 - val_accuracy: 0.7383
Epoch 3/100
36/36 - 0s - loss: 0.4117 - accuracy: 0.7924 - val_loss: 0.4403 - val_accuracy: 0.7405
Epoch 4/100
36/36 - 0s - loss: 0.3938 - accuracy: 0.7985 - val_loss: 0.4245 - val_accuracy: 0.7494
Epoch 5/100
36/36 - 0s - loss: 0.3830 - accuracy: 0.8050 - val_loss: 0.4209 - val_accuracy: 0.7539
Epoch 6/100
36/36 - 0s - loss: 0.3754 - accuracy: 0.8075 - val_loss: 0.4180 - val_accuracy: 0.7494
Epoch 7/100
36/36 - 0s - loss: 0.3685 - accuracy: 0.8134 - val_loss: 0.4133 - val_accuracy: 0.7606
Epoch 8/100
36/36 - 0s - loss: 0.3687 - accuracy: 0.8083 - val_loss: 0.4002 - val_accuracy: 0.7808
Epoch 9/100
36/36 - 0s - loss: 0.3613 - accuracy: 0.8120 - val_loss: 0.4010 - val_accuracy: 0.7718
Epoch 10/100
36/36 - 0s - loss: 0.3598 - accuracy: 0.8111 - val_loss: 0.4022 - val_accuracy: 0.7673


## Testing the model



In [19]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3206 - accuracy: 0.8527


In [20]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.32. Test accuracy: 85.27%
